In [ ]:
import altair as alt
import pandas as pd
from omegaconf import OmegaConf

from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

import _report_visuals_utils
import viz_data_prep

readable_dict = OmegaConf.load("readable2.yml")

In [ ]:
FILE = GTFS_DATA_DICT.digest_tables.route_schedule_vp

# some of the portfolio grain can be dealt with
# but separate out the renaming/replacing/subsetting to separate script

df = pd.read_parquet(
    f"{RT_SCHED_GCS}{FILE}.parquet",
    filters = [[
        ("portfolio_organization_name", "==", "City of West Hollywood"),
        ("recent_combined_name", "in", ["Cityline Local-East", "Cityline Local-West"])
    ]]
).pipe(
    viz_data_prep.data_wrangling_for_visualizing,
    viz_data_prep.route_direction_cols_for_viz,
    viz_data_prep.readable_col_names
)

In [ ]:
from IPython.display import HTML, display

# Set drop down menu to be on the upper right for the charts
display(
    HTML(
        """
<style>
form.vega-bindings {
  position: absolute;
  right: 0px;
  top: 0px;
}
</style>
"""
    )
)

In [ ]:
def route_filter(df):
    routes_list = df["Route"].unique().tolist()

    route_dropdown = alt.binding_select(
        options=routes_list,
        name="Routes: ",
    )
    # Column that controls the bar charts
    xcol_param = alt.selection_point(
        fields=["Route"], value=routes_list[0], bind=route_dropdown
    )
    
    
    chart1 = (
        _report_visuals_utils.sample_spatial_accuracy_chart(df[df.Period=="All Day"])
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    ) 

    chart2 = (
        _report_visuals_utils.sample_avg_scheduled_min_chart(df[df.Period=="All Day"])
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    
    )
    
    chart_list = [
        #chart1,
        chart2
    ]
    chart = alt.vconcat(*chart_list)

    
    return chart
 

In [ ]:
route_filter(df)